<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [11]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [12]:
df.shape

(7043, 21)

In [13]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [14]:
# cleaning the TotalCharges column, turning it into a float column it was born to be
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df['TotalCharges'] = df['TotalCharges'].fillna(method='ffill')

In [15]:
# Split numerics from categoricals, remove id column, train test split
from sklearn.model_selection import train_test_split

target = 'Churn'
index = 'customerID'
features = [i for i in df.columns if i not in [target, index]]

# get numeric and categorical column names so we can treat each sepeartely later.
numerics = df.select_dtypes(include=np.number).columns.tolist()
categoricals = df.select_dtypes(exclude=np.number).columns.drop([index, target]).tolist()

# split X and y
y_train = df[target]
X_train = df[features]

# convert output column to numbers from Yes and No.
convert_to_binary_map = {'Yes': 1, 'No': 0}
y_train = y_train.map(convert_to_binary_map)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=2019)

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False)

# normalize the numeric data
X_train = X_train.copy()
X_test = X_test.copy()
X_train[numerics] = scaler.fit_transform(X_train[numerics])
X_test[numerics] = scaler.transform(X_test[numerics])
X_train_scaled = X_train[numerics].to_numpy()
X_test_scaled = X_test[numerics].to_numpy()

# one hot encode the categorical data
X_train_cat_encoded = encoder.fit_transform(X_train[categoricals])
X_test_cat_encoded = encoder.transform(X_test[categoricals])

# combine the two
X_train_mat = np.concatenate((X_train_scaled, X_train_cat_encoded), axis=1)
X_test_mat = np.concatenate((X_test_scaled, X_test_cat_encoded), axis=1)

In [63]:
X_train_mat.shape, X_test_mat.shape

((5282, 45), (1761, 45))

In [67]:
model.fit(X_train_mat, y_train.to_numpy(), epochs=20, verbose=0)

In [123]:
X_test.shape, y_test.shape

((1761, 19), (1761,))

In [124]:
X_train.shape, y_train.shape

((5282, 6538), (5282,))

In [68]:
scores = model.evaluate(X_test_mat, y_test)
scores

1761/1761 [==============================] - 0s 101us/sample - loss: 0.4161 - acc: 0.8075


[0.41613934932817104, 0.8074957]

In [71]:
# Now we begin hyperparameter optimization
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# KerasClassifier needs a model in this form to work
def create_model():

    input_dim = X_train_mat.shape[1]

    model = Sequential(name='ModelTry2')
    model.add(Dense(3, input_dim=input_dim, activation='relu', name='InputLayer'))
    model.add(Dense(4, activation='relu', name='HiddenLayer1'))
    model.add(Dense(1, activation='sigmoid', name='OutputLayer'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
    'epochs': [20]
}

grid = GridSearchCV(estimator=model, cv=3, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_mat, y_train)

In [73]:
print(f"{grid_result.best_score_} using {grid_result.best_params_}")

0.8010223305098986 using {'batch_size': 20, 'epochs': 20}


In [76]:
# param_grid = {
#     'batch_size': [10, 20],
# #     'lr': [.001, .01, .1, .2, .3, .5],
#     'epochs': [20],
# }

# grid = GridSearchCV(estimator=model, cv=3, param_grid=param_grid, n_jobs=1)
# grid_result = grid.fit(X_train_mat, y_train)

ValueError: lr is not a legal parameter

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?